# Text Generation with Large Language Models (LLMs)

In this notebook, we will explore how to use a pre-trained LLM for text generation. Specifically, we will use GPT-2 to generate coherent and contextually relevant text based on an input prompt. Through this notebook, we will:
- Load a pre-trained LLM and its tokenizer.
- Define a function to generate text using the model.
- Experiment with different parameters to influence the generated text.

## Setup: Import Libraries and Set Up the Environment

First and foremost, one must import the necessary libraries and set up the environment. In this particular case, we will use the `transformers` library from Hugging Face, which provides a simple interface for working with pre-trained models.

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# Check if a GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


## Load Model and Tokenizer

Then, we load the pre-trained GPT-2 model and its corresponding tokenizer. The tokenizer is responsible for converting text into the format that the model can understand, and vice versa.

In [2]:
# Load the pre-trained GPT-2 model and its tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Move the model to the appropriate device
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

## Text Generation Function

Here, we define a function that takes a text prompt as input and generates a continuation using the GPT-2 model. The function allows for customization of various parameters such as the maximum length of the generated text and the number of different sequences to generate.

In [3]:
def generate_text(prompt, max_length=50, num_return_sequences=1, temperature=1.0, top_p=1.0, top_k=50):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)    # tokenize and prepare the prompt for the model
    
    outputs = model.generate(    # generate text using the model
        inputs.input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        repetition_penalty=1.5,
        do_sample=True,    # allows sampling and diversity in output
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        pad_token_id=tokenizer.eos_token_id)    # prevents warnings
    
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]    # decode and return the generated text
    return generated_texts

## Run Examples: Generate and Display Text

Let's test our text generation function with a few example prompts. We will input a starting sentence, and the model will generate the rest of the text.

In [4]:
prompt = "Once upon a time in a land far away"
generated_texts = generate_text(prompt, max_length=50, num_return_sequences=3)

for i, text in enumerate(generated_texts):
    print(f"Generated Text {i+1}:\n{text}\n")

Generated Text 1:
Once upon a time in a land far away I became known as something of an alien, but for whatever reason it's so rare these days to see myself or be able even one bit like me because they don't know how and if anyone can possibly

Generated Text 2:
Once upon a time in a land far away, humans were able to do whatever they needed without them knowing what it was like. At one point the world collapsed into chaos before all of humanity could realize that this planet as we know itself lacked natural resources

Generated Text 3:
Once upon a time in a land far away from us, we all took part. If he had to die for what others have brought… then I wonder if that is it?" Merely being told the truth she looked up at Dumbledore's lips



## Experimentation & Exploring Different Parameters

Experiment with different prompts and parameters like `max_length`, `temperature`, and `top_k` to see how they affect the generated text. By tweaking these, one can influence the creativity and coherence of the generated content.

In [6]:
experiment_prompt = "In a futuristic world where AI controls everything"
experiment_texts = generate_text(experiment_prompt, max_length=100, num_return_sequences=2, temperature=0.7, top_p=0.9)

for i, text in enumerate(experiment_texts):
    print(f"Experiment {i+1}:\n{text}\n")

Experiment 1:
In a futuristic world where AI controls everything, there are no rules. Humans have to follow the laws of physics; their bodies cannot move at will and they must be restrained from touching each other (but not touch me).
I am one of those who believe that this is possible only with science-based solutions! If you're willing - if your body can control it's going down without any problems because all we need is an understanding or something like "we've got nothing left". But what about

Experiment 2:
In a futuristic world where AI controls everything from cars to food, the human brain is able only in emergencies.
 (Photo: Marvel Comics) Story Highlights The characters are created by Scott Snyder and Mark Millar with help of Joe Quesada as well Assemble Your Own Brain - "The Avengers" series focuses on an unlikely team who have been forced into battle against their own superpowers This week's issue will feature one-shots featuring some familiar faces including Thor, Hulk, XO

## Enhanced Function

In [29]:
def generate_text(prompt, max_length=200, temperature=0.9, top_p=0.95, top_k=60, num_return_sequences=3, num_beams=7):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    outputs = model.generate(
        inputs.input_ids,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2,
        repetition_penalty=1.5,
        top_k=top_k,
        top_p=top_p,
        temperature=temperature,
        num_beams=num_beams,  # beam search
        early_stopping=True,
        pad_token_id=tokenizer.eos_token_id)
    
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts

In [30]:
prompt = "The future of technology is filled with both incredible possibilities and unforeseen challenges. Scientists and engineers around the world are working on innovations that could change our lives forever. One such innovation is"
generated_texts = generate_text(prompt, max_length=200, temperature=0.9, top_p=0.95, top_k=60, num_return_sequences=3)

for i, text in enumerate(generated_texts):
    print(f"Output {i+1}:\n{text}\n")

Output 1:
The future of technology is filled with both incredible possibilities and unforeseen challenges. Scientists and engineers around the world are working on innovations that could change our lives forever. One such innovation is artificial intelligence, or AI.

AI is a new form of machine learning that can be used to solve complex problems. It has the potential to revolutionize the way we think about information and how we interact with it. In this article, we will look at some of the most exciting developments in AI and its potential applications.

Output 2:
The future of technology is filled with both incredible possibilities and unforeseen challenges. Scientists and engineers around the world are working on innovations that could change our lives forever. One such innovation is artificial intelligence, or AI.

AI is a new form of machine learning that can be used to solve complex problems. It has the potential to revolutionize the way we think about information and how we int

## Enhancing the Output

Improving the coherence and quality of the generated text from a large language model involves fine-tuning various aspects of the model's output settings, and possibly even the model itself. Some strategies are:

1. Adjust Sampling Parameters

    • **Temperature:** Lowering the temperature value (e.g., between 0.7 to 0.9) can make the text more focused and deterministic, leading to more coherent sentences. However, too low a temperature (e.g., below 0.5) might result in repetitive or overly conservative text.

    • **Top-p (Nucleus Sampling):** Top-p sampling (with a value of, e.g., 0.9) considers the smallest set of tokens whose cumulative probability adds up to 0.9. This filters out unlikely options and keeps the text more coherent.

    • **Top-k:** Top-k sampling (with a value of, e.g., 50) restricts the number of tokens considered at each step to the top 50, which can help maintain coherence by focusing on the most likely options.
    

2. Increase the Context Size

    • **Longer prompts:** Providing a more detailed prompt or context can guide the model to generate more contextually relevant texts. Instead of starting with a short phrase, you could give a paragraph that sets the scene or introduces key concepts.

    • **Sliding window approach:** For generating longer texts, consider feeding the model its previous outputs as part of the prompt in subsequent iterations.


3. Fine-Tune the Model on Specific Data

    • **Domain-specific fine-tuning:** For specific types of content with high coherence (e.g., technical writing, storytelling), fine-tune the pre-trained model on a dataset that matches desired output style can significantly improve results.

    • **Training with Reinforcement Learning:** Techniques like Reinforcement Learning from Human Feedback (RLHF) can be used to adjust the model's behavior based on specific quality metrics.


4. Iterative Prompting

    • **Iterative prompt refinement:** After generating text, modify the prompt based on the output received and rerun the generation.


5. Post-Processing the Generated Text

    • **Text editing:** After generating text, you can apply rules or heuristics to clean up or adjust the output. This might include correcting grammar, removing contradictions, or merging sentences for better flow.

    • **Chaining outputs:** Generate multiple completions and then manually or programmatically select and combine the best parts. This allows the construction of a more coherent narrative.


6. Experiment with Beam Search

    • **Beam Search:** Beam search (num_beams=>3) is a technique where the model explores multiple paths before finalizing on the most coherent one. This often leads to better-structured sentences and paragraphs.


7. Use Larger or Newer Models

    • **Try GPT-3 or GPT-4:** Consider using GPT-3 or GPT-4, which are more advanced and capable of generating contextually appropriate text.

## Next Steps...

- Fine-tuning the model on a custom dataset to generate domain-specific text.
- Exploring more advanced models like GPT-3 or GPT-4.
- Implementing a user interface for interactive text generation.